#### **DDQN with Hyperparameter setting**

This is an example of Hyperparameter setting for CARTPOLE. <br>
With the actual setup it can take many hours (even with a GPU) <br>
Press the colab icon to run it in COLAB (in this way you save your laptop for other tasks)

<a target="_blank" href="https://colab.research.google.com/github/castorgit/RL_course/blob/main/00_LunarLander-COLAB_render.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [1]:
import tensorflow as tf
import keras as k
print(k.__version__)
print(tf.__version__)

2024-12-22 09:03:38.984694: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-22 09:03:39.005671: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-22 09:03:39.028796: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-22 09:03:39.035193: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-22 09:03:39.052362: I tensorflow/core/platform/cpu_feature_guar

3.6.0
2.17.1


In [2]:
#### **Install Packages**
!pip install gymnasium[box2d]
!pip install numpy
!pip install random
!pip install matplotlib
!pip install tensorflow==2.17.1
!pip install keras==3.6.0
!pip pandas as pd

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [3]:
# Imports
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow warnings
import gymnasium as gym
import numpy as np
import random
import matplotlib.pyplot as plt
from collections import deque
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import time
from PIL import Image
from sklearn.model_selection import ParameterGrid
import pandas as pd

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


I0000 00:00:1734854625.895886  898937 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734854625.927105  898937 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734854625.927192  898937 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


#### **Environment Setup**

In [4]:
env = gym.make("CartPole-v1")
state_size = env.observation_space.shape[0]  # 8 state variables
action_size = env.action_space.n  # 4 discrete actions
tf.random.set_seed(221)

#### **hyperparameter search space**

In [5]:
param_grid = {
    'learning_rate': [0.0001],
    'gamma': [0.99],
    'batch_size': [128],
    'tau': [0.1],
    'epsilon_decay': [0.99],
    'retrain_steps': [10],
    'initial_lr' : [0.001, 0.01, 0.1],
    'd_steps'    : [10000, 50000, 5000],
    'd_rate'     : [0.99, 0.98, 0.97]
    
}

In [6]:
#learning_rate = 0.0001
gamma = 0.98
batch_size = 128
epsilon_start = 1.0
epsilon_min = 0.01
epsilon_decay = 0.99
tau = 0.15                         # For soft target network update
buffer_capacity = 10000
max_episodes = 100                   # We train only 50 to see how the agent learns
max_steps = 1200
solved_threshold = 200
verbose = 0                        # 0: No trace 1: Trace

d_steps = 10000
d_rate  = 0.99

In [7]:
# Replay buffer
replay_buffer = deque(maxlen=buffer_capacity)

# Add experience to replay buffer
def store_experience(state, action, reward, next_state, done):
    replay_buffer.append((state, action, reward, next_state, done))

# Sample experiences from the replay buffer
def sample_experiences(batch_size):
    indices = np.random.choice(len(replay_buffer), batch_size, replace=False)
    batch = [replay_buffer[i] for i in indices]
    states, actions, rewards, next_states, dones = zip(*batch)
    
    return (
        np.vstack(states),
        np.array(actions),
        np.array(rewards),
        np.vstack(next_states),
        np.array(dones, dtype=np.float32)
    )

#### **Neural Network definition**

In [8]:
# Build the neural network model
def build_model(state_size, action_size):
    inputs = Input(shape=(state_size,))  
    x = Dense(16, activation="relu")(inputs)
#    x = Dropout(0.3)(x)
    x = Dense(64, activation="relu")(x)
#    x = Dropout(0.3)(x)
    x = Dense(16, activation="relu")(x)
    outputs = Dense(action_size, activation="linear")(x)
    model = Model(inputs=inputs, outputs=outputs)
    
    
    lr_schedule = ExponentialDecay(
                            initial_learning_rate=initial_lr,
                            decay_steps = d_steps,
                            decay_rate  = d_rate,
                            staircase=True)                                  
    
    model.compile(optimizer=Adam(learning_rate=lr_schedule), loss="mse")
    return model

#### **Support Functions**

In [9]:
# Soft update function for the target network
def soft_update(model, target_model, tau):
    target_weights = target_model.get_weights()
    model_weights = model.get_weights()
    new_weights = [
        tau * mw + (1 - tau) * tw for mw, tw in zip(model_weights, target_weights)
    ]
    target_model.set_weights(new_weights)

# Polski Optimization Function
def polski_optimization(weights, beta=0.01):
    return [w * (1 - beta) for w in weights]

# Double DQN target calculation
def experience_replay_with_ddqn(model, target_model, batch_size, gamma, tau, step):
    if len(replay_buffer) < batch_size:
        return

    states, actions, rewards, next_states, dones = sample_experiences(batch_size)

    # Predict Q-values for next states using both networks
    next_q_values = model.predict(next_states, verbose=0)
    best_actions = np.argmax(next_q_values, axis=1)
    target_q_values = target_model.predict(next_states, verbose=0)

    # Update Q-values using Double DQN formula
    targets = rewards + gamma * target_q_values[np.arange(batch_size), best_actions] * (1 - dones)

    # Update main Q-network
    q_values = model.predict(states, verbose=0)
    q_values[np.arange(batch_size), actions] = targets
    model.fit(states, q_values, epochs=1, verbose=0)

    # Apply soft update to target network
    if step % retrain_steps == 0:   
        soft_update(model, target_model, tau)

In [10]:
# Initialize models
#model = build_model(state_size, action_size)
#target_model = build_model(state_size, action_size)
#target_model.set_weights(model.get_weights())  # Sync target network initially


In [11]:
# Training loop
def train_ddqn_with_params(params, max_episodes, verbose):
    
    global initial_lr, gamma, batch_size, tau, epsilon_decay, retrain_steps
    initial_lr = params['initial_lr']
    gamma = params['gamma']
    batch_size = params['batch_size']
    tau = params['tau']
    epsilon_decay = params['epsilon_decay']
    retrain_steps = params['retrain_steps']
    d_steps = params['d_steps']
    d_rate = params['d_rate']
    
    print('**************************************************************')
    print('ini_lr:', initial_lr, 'gamma', gamma, 'batch_size', batch_size, 'tau ', tau, 'epsilon decay', epsilon_decay,
                 'retrain_steps ', retrain_steps, 'd_steps', d_steps, 'd_rate', d_rate)
    
    # Initialize models
    model = build_model(state_size, action_size)
    target_model = build_model(state_size, action_size)
    target_model.set_weights(model.get_weights())  # Sync target network initially
    

    epsilon = epsilon_start
    episode_rewards = []
    rolling_avg_rewards = []

    start_time = time.time()

    for episode in range(max_episodes):
        state, _ = env.reset()
        state = np.reshape(state, [1, state_size])
        total_reward = 0

        for step in range(max_steps):
            # Epsilon-greedy policy
            if np.random.rand() <= epsilon:
                action = np.random.randint(action_size)  # Explore
            else:
                action_vals = model.predict(state, verbose=0)
                action = np.argmax(action_vals[0])  # Exploit

            # Perform action
            next_state, reward, done, _, _ = env.step(action)
            next_state = np.reshape(next_state, [1, state_size])
            total_reward += reward

            # Store experience
            store_experience(state, action, reward, next_state, done)

            # Update state
            state = next_state

            # Train using experience replay
            experience_replay_with_ddqn(model, target_model, batch_size, gamma, tau, step)

            if done:
                break

        # Decay epsilon
        epsilon = max(epsilon_min, epsilon * epsilon_decay)

        # Record reward
        episode_rewards.append(total_reward)
        rolling_avg = np.mean(episode_rewards[-20:])
        rolling_avg_rewards.append(rolling_avg)

        # Print progress
        if verbose:
           print(f"Episode: {episode+1:3}/{max_episodes}, Reward: {total_reward:+7.2f}, "
                f"Epsilon: {epsilon:.2f}, Rolling Avg: {rolling_avg:4.2f}, Steps: {step:3}")

        # Check if environment is solved
        if rolling_avg >= solved_threshold:
            print(f"Environment solved in {episode+1} episodes!")
            model.save("XXX_ddqn_model1.keras")
            break

    end_time = time.time()
    print(f"Rewards Rolling Avg: {rolling_avg:4.2f}")
    print(f"Training completed in {(end_time - start_time)/60:.2f} minutes")
    print(initial_lr, d_steps, d_rate, gamma, batch_size, tau, epsilon_decay, retrain_steps)
    
    return (rolling_avg)

In [12]:
# Perform grid search
results = []
for params in ParameterGrid(param_grid):
    avg_reward = train_ddqn_with_params(params, max_episodes, verbose)
    results.append({**params, 'avg_reward': avg_reward})

# Convert results to DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='avg_reward', ascending=False)

# Display top results
print("Top Hyperparameters:")
print(results_df.head())

**************************************************************
ini_lr: 0.001 gamma 0.99 batch_size 128 tau  0.1 epsilon decay 0.99 retrain_steps  10 d_steps 10000 d_rate 0.99


I0000 00:00:1734854626.046520  898937 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734854626.046752  898937 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734854626.046803  898937 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734854627.817360  898937 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734854627.817519  898937 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-22

Rewards Rolling Avg: 81.65
Training completed in 19.94 minutes
0.001 10000 0.99 0.99 128 0.1 0.99 10
**************************************************************
ini_lr: 0.01 gamma 0.99 batch_size 128 tau  0.1 epsilon decay 0.99 retrain_steps  10 d_steps 10000 d_rate 0.99
Rewards Rolling Avg: 35.35
Training completed in 15.17 minutes
0.01 10000 0.99 0.99 128 0.1 0.99 10
**************************************************************
ini_lr: 0.1 gamma 0.99 batch_size 128 tau  0.1 epsilon decay 0.99 retrain_steps  10 d_steps 10000 d_rate 0.99
Rewards Rolling Avg: 19.50
Training completed in 9.53 minutes
0.1 10000 0.99 0.99 128 0.1 0.99 10
**************************************************************
ini_lr: 0.001 gamma 0.99 batch_size 128 tau  0.1 epsilon decay 0.99 retrain_steps  10 d_steps 50000 d_rate 0.99
Rewards Rolling Avg: 57.70
Training completed in 24.40 minutes
0.001 50000 0.99 0.99 128 0.1 0.99 10
**************************************************************
ini_lr: 0.01 ga

In [13]:
import session_info
session_info.show(html=False)

-----
PIL                 11.0.0
gymnasium           1.0.0
keras               3.6.0
matplotlib          3.9.2
numpy               1.26.4
pandas              2.2.3
session_info        1.0.0
sklearn             1.5.2
tensorflow          2.17.1
-----
IPython             8.28.0
jupyter_client      8.6.3
jupyter_core        5.7.2
-----
Python 3.12.3 (main, Nov  6 2024, 18:32:19) [GCC 13.2.0]
Linux-5.15.167.4-microsoft-standard-WSL2-x86_64-with-glibc2.39
-----
Session information updated at 2024-12-22 14:38


#### **Some rules of thumb**

Soft Updates Every Step: Use a small tau (e.g., 0.005).<br>
Soft Updates Every N Steps/Episodes: Allows a higher tau (e.g., 0.1).<br>
Hard Updates: Perform less frequently (e.g., every 1000 steps) target_model.set_weights(model.get_weights())